# DeepEval demo en Binder usando otro modelo como output

Tomamos el modelo gpt-40-mini como generador de output, es decir, es el modelo evaluado.

> **Sugerencia**: Si estás en Binder, este repo debe incluir un `requirements.txt` (ya preparado) para que el entorno se construya automáticamente.

In [2]:

# === Instalación/Comprobación de dependencias ===
import importlib, sys, subprocess

def pip_install(*packages):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', '-U', *packages])

try:
    import deepeval  # noqa
except Exception:
    pip_install('deepeval', 'python-dotenv')

print('Entorno listo ✅')


Entorno listo ✅


In [3]:
from getpass import getpass
import os

os.environ ["OPENAI_API_KEY"] = getpass("Pega aquí tu clave API: ")

Pega aquí tu clave API:  ········


In [10]:

from openai import OpenAI
client = OpenAI()

sut_model = "gpt-4o-mini"  # o tu modelo fine-tuned

inputs = [
    "¿Qué pasa si estos zapatos no me quedan?",
    "¿Que ingredientes necesito para hacer tortilla española?"
]

expected_outputs = [
    "Tienes derecho a reembolso dentro de 30 días.",
    "Necesitarás huevos, patatas y cebolla"
]

# Generar una salida por input (evaluación independiente)
actual_outputs = []
for text in inputs:
    resp = client.responses.create(
        model=sut_model,
        input=[{"role": "user", "content": text}]
    )
    actual_outputs.append(resp.output_text)

# Visualizar resultados del SUT
for i, (inp, out, gold) in enumerate(zip(inputs, actual_outputs, expected_outputs), start=1):
    print(f"\n--- Caso {i} ---")
    print("Input:", inp)
    print("Output (SUT):", out)
    print ("Golden Answer:" ,gold)





--- Caso 1 ---
Input: ¿Qué pasa si estos zapatos no me quedan?
Output (SUT): Si los zapatos no te quedan, hay varias opciones que puedes considerar:

1. **Devolución o cambio**: Si compraste los zapatos recientemente, verifica la política de devolución de la tienda. Muchas permiten cambios o devoluciones si el producto no se ajusta.

2. **Ajuste**: Algunas veces, un zapatero profesional puede modificar los zapatos para que se ajusten mejor a tus pies.

3. **Uso de plantillas**: Si el problema es que están un poco grandes, podrías usar plantillas para ajustarlos mejor.

4. **Reventa**: Considera venderlos si ya no puedes devolverlos. Hay muchas plataformas donde puedes vender zapatos nuevos o poco usados.

5. **Regalar**: Si no los necesitas, también puedes considerar regalarlos a alguien que sí les sirva.

Siempre es bueno probar los zapatos antes de comprarlos. Asegúrate de caminar un poco con ellos en la tienda para verificar el ajuste.
Golden Answer: Tienes derecho a reembolso dent

## Métricas con **LLM** (requiere API key)
Las métricas como `AnswerRelevancyMetric` o `GEVal` usan un **LLM como juez**. 
Por defecto DeepEval utiliza modelos de OpenAI; configura `OPENAI_API_KEY` (o usa otro proveedor soportado) antes de ejecutar.

In [21]:

# Ejemplo con AnswerRelevancyMetric
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase
from deepeval import evaluate
from openai import OpenAI
import time
client = OpenAI()

time.sleep(60) 

sut_model = "gpt-4o-mini"  # o tu modelo fine-tuned

#Definimos los inputs:

inputs = [
    "¿Qué pasa si estos zapatos no me quedan?",
    "¿Que ingredientes necesito para hacer tortilla española?"
]


# Generar una salida por input (evaluación independiente)
actual_outputs = []
for text in inputs:
    resp = client.responses.create(
        model=sut_model,
        input=[{"role": "user", "content": text}]
    )
    actual_outputs.append(resp.output_text)

time.sleep(60)

# Crea los test cases (uno por input)
test_cases = []
for i, (inp, out) in enumerate(zip(inputs, actual_outputs), start=1):
    tc = LLMTestCase(
        input=inp,
        actual_output=out,
        
    )
    test_cases.append(tc)

time.sleep(60)

# El modelo juez (Evaluation Model) para las métricas LLM-as-a-judge
judge_model = "gpt-4o-mini"   # elige el modelo para juzgar (puede ser otro más económico)

# Métricas
answer_relevancy = AnswerRelevancyMetric(
    threshold=0.7,
    model=judge_model,
    include_reason=False
)


_ = evaluate(test_cases, [answer_relevancy])


✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o-mini, strict=False, async_mode=True)...

Output()



Metrics Summary

  - ✅ Answer Relevancy (score: 0.8888888888888888, threshold: 0.7, strict: False, evaluation model: gpt-4o-mini, reason: None, error: None)

For test case:

  - input: ¿Que ingredientes necesito para hacer tortilla española?
  - actual output: Para hacer una tortilla española, necesitarás los siguientes ingredientes:

1. **Patatas**: 3-4 medianas
2. **Huevos**: 4-6 (dependiendo del tamaño de la tortilla)
3. **Cebolla** (opcional): 1 mediana
4. **Aceite de oliva**: suficiente para freír
5. **Sal**: al gusto

Estos son los ingredientes básicos. Puedes ajustar las cantidades según el tamaño de la tortilla que desees preparar. ¡Buen provecho!
  - expected output: None
  - context: None
  - retrieval context: None


Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-4o-mini, reason: None, error: None)

For test case:

  - input: ¿Qué pasa si estos zapatos no me quedan?
  - actual output: Si los zapatos no te quedan, h

⚠ WARNING: No hyperparameters logged.
» ]8;id=258505;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 8.52s | token cost: 0.0007392 USD)
» Test Results (2 total tests):
   » Pass Rate: 100.0% | Passed: 2 | Failed: 0

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.